In [ ]:
%load_ext autoreload
%autoreload 2
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=2)
%matplotlib inline

In [ ]:
# get data
from eden.converter.graph.gspan import gspan_to_eden
from itertools import islice

def get_graphs(dataset_fname='../toolsdata/bursi.pos.gspan', size=100):
    return  islice(gspan_to_eden(dataset_fname),size)


In [ ]:
'''
the preprocessor will learn the abstraction. 
'''
from graphlearn.utils import draw
import graphlearn.abstract_graphs.learned as learn
from eden.graph import Vectorizer

#preparing
v=Vectorizer(triangular_decomposition=False,complexity=3)
pp=learn.PreProcessor(kmeans_clusters=4)
pp.set_param(v)

# the magic happens here
wrappers=pp.fit_transform(get_graphs(size=100))

#lets see what we did there
for i in range(2):
    draw.graphlearn([wrappers[i+5].graph(),wrappers[i+5].base_graph(),wrappers[i+5].abstract_graph()],
                    size=13,
                    contract=True, 
                    abstract_color='red',
                    vertex_label='label'
                   )



In [ ]:
'''
lets see if these wrappers give us CIPS as this is their only purpose :)
'''
radius_list=[0,2]
thickness_list=[2,4]
base_thickness_list=[2]
cips=wrappers[0].all_core_interface_pairs(thickness_list=[2],radius_list=[0,1],hash_bitmask=2**20-1)
draw.graphlearn(cips[0][0].graph, contract=False)
draw.graphlearn(cips[0][1].graph, contract=False)


In [ ]:
%%time
'''
wrappers work, now we can see if we can do sampling. 
'''
import graphlearn.abstract_graphs.learned as learn
from graphlearn.graphlearn import Sampler as GLS
graphs = get_graphs(size=200)
sampler=GLS(radius_list=[0,1],
            thickness_list=[1], 
            min_cip_count=2, 
            min_interface_count=2, 
            preprocessor=learn.PreProcessor(base_thickness_list=[2]))

sampler.fit(graphs,grammar_n_jobs=1)
print 'done'



In [ ]:
#draw production rules
draw.draw_grammar(sampler.lsgg.productions,n_productions=5,n_graphs_per_production=5,
                     n_graphs_per_line=5, size=9, contract=False,
                     colormap='Paired', invert_colormap=False,node_border=1,
                     vertex_alpha=0.6, edge_alpha=0.5, node_size=450, abstract_interface=True)

    

In [ ]:
%%time
'''
Molecule sampling
'''
import os
os.nice(19)
import graphlearn.utils.draw as draw
import graphlearn.abstract_graphs.molecules_circle as mole
import itertools
graphs = get_graphs()
id_start=15
id_end=id_start+9
graphs = itertools.islice(graphs,id_start,id_end)
n_steps=50

graphs = sampler.sample(graphs,
                        n_samples=5,
                        batch_size=1,
                        n_steps=n_steps,
                        n_jobs=1,
                        quick_skip_orig_cip=True,
                        probabilistic_core_choice=False,
                        burnin=0,
                        improving_threshold=0.5,
                        select_cip_max_tries=100,
                        keep_duplicates=True,
                        include_seed=True)

 
scores=[]
ids=range(id_start,id_end)
for i,path_graphs in enumerate(graphs):
    print 'Graph id: %d'%(ids[i])
    scores.append(sampler.monitors[i].sampling_info['score_history'])
    
    draw.graphlearn(path_graphs,
                           n_graphs_per_line=5, size=10, 
                           colormap='Paired', invert_colormap=False,node_border=0.5, vertex_color='color_level',
                           vertex_alpha=0.5, edge_alpha=0.7, node_size=450,edge_label='label'
                   )
    

In [ ]:
%matplotlib inline
from itertools import islice
import numpy as np
import matplotlib.pyplot as plt
step=1
num_graphs_per_plot=3
num_plots=np.ceil([len(scores)/num_graphs_per_plot])
for i in range(num_plots):
    plt.figure(figsize=(10,5))
    for j,score in enumerate(scores[i*num_graphs_per_plot:i*num_graphs_per_plot+num_graphs_per_plot]):
        data = list(islice(score,None, None, step))
        plt.plot(data, label='graph %d'%(j+i*num_graphs_per_plot+id_start))
    plt.legend(loc='lower right')
    plt.grid()
    plt.ylim(-0.1,1.1)
    plt.show()